In [22]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import copy
%matplotlib inline


In [23]:
data = pd.read_csv('./data/nbaData.csv')

In [24]:
data

,#,Season Start,Player Name,Player Salary in $,Pos,Age,Tm,G,GS,MP,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,8035.0,1986.0,A.C. Green,NaN,PF,22.0,LAL,82.0,1.0,1542.0,...,61.1%,160.0,221.0,381.0,54.0,49.0,49.0,99.0,229.0,521.0
1,8420.0,1987.0,A.C. Green,NaN,PF,23.0,LAL,79.0,72.0,2240.0,...,78.0%,210.0,405.0,615.0,84.0,70.0,80.0,102.0,171.0,852.0
2,8807.0,1988.0,A.C. Green,NaN,PF,24.0,LAL,82.0,64.0,2636.0,...,77.3%,245.0,465.0,710.0,93.0,87.0,45.0,120.0,204.0,937.0
3,9242.0,1989.0,A.C. Green,NaN,PF,25.0,LAL,82.0,82.0,2510.0,...,78.6%,258.0,481.0,739.0,103.0,94.0,55.0,119.0,172.0,1088.0
4,9688.0,1990.0,A.C. Green,"$1,750,000.00",PF,26.0,LAL,82.0,82.0,2709.0,...,75.1%,262.0,450.0,712.0,90.0,66.0,50.0,116.0,207.0,1061.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24620,19003.0,2008.0,Zydrunas Ilgauskas,"$10,841,615.00",C,32.0,CLE,73.0,73.0,2222.0,...,80.2%,263.0,419.0,682.0,104.0,34.0,120.0,135.0,247.0,1029.0
24621,19600.0,2009.0,Zydrunas Ilgauskas,"$11,541,074.00",C,33.0,CLE,65.0,65.0,1765.0,...,79.9%,157.0,333.0,490.0,64.0,28.0,84.0,90.0,183.0,838.0
24622,20187.0,2010.0,Zydrunas Ilgauskas,NaN,C,34.0,CLE,64.0,6.0,1339.0,...,74.3%,114.0,231.0,345.0,48.0,14.0,50.0,63.0,183.0,474.0
24623,20802.0,2011.0,Zydrunas Ilgauskas,NaN,C,35.0,MIA,72.0,51.0,1145.0,...,78.3%,108.0,179.0,287.0,26.0,23.0,58.0,52.0,185.0,360.0


In [25]:
print(data.columns)

Index(['#', 'Season Start', 'Player Name', ' Player Salary in $ ', 'Pos',
       'Age', 'Tm', 'G', 'GS', 'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%',
       'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'blanl', 'OWS',
       'DWS', 'WS', 'WS/48', 'blank2', 'OBPM', 'DBPM', 'BPM', 'VORP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS'],
      dtype='object')


In [26]:
# droping the columns with blank data
data = data.drop(columns=['blanl', 'blank2'])

In [27]:
print(data.columns)

Index(['#', 'Season Start', 'Player Name', ' Player Salary in $ ', 'Pos',
       'Age', 'Tm', 'G', 'GS', 'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%',
       'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS',
       'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')


In [28]:
copyData = copy.deepcopy(data)

In [64]:
totalOrigRows = len(copyData)
print(totalOrigRows)

24625


In [65]:
numRemainCols = len(copyData.columns)
print(numRemainCols)

52


In [68]:
data = copyData.dropna(thresh=numRemainCols-1)

In [69]:
data

,#,Season Start,Player Name,Player Salary in $,Pos,Age,Tm,G,GS,MP,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,8035.0,1986.0,A.C. Green,NaN,PF,22.0,LAL,82.0,1.0,1542.0,...,61.1%,160.0,221.0,381.0,54.0,49.0,49.0,99.0,229.0,521.0
1,8420.0,1987.0,A.C. Green,NaN,PF,23.0,LAL,79.0,72.0,2240.0,...,78.0%,210.0,405.0,615.0,84.0,70.0,80.0,102.0,171.0,852.0
2,8807.0,1988.0,A.C. Green,NaN,PF,24.0,LAL,82.0,64.0,2636.0,...,77.3%,245.0,465.0,710.0,93.0,87.0,45.0,120.0,204.0,937.0
3,9242.0,1989.0,A.C. Green,NaN,PF,25.0,LAL,82.0,82.0,2510.0,...,78.6%,258.0,481.0,739.0,103.0,94.0,55.0,119.0,172.0,1088.0
4,9688.0,1990.0,A.C. Green,"$1,750,000.00",PF,26.0,LAL,82.0,82.0,2709.0,...,75.1%,262.0,450.0,712.0,90.0,66.0,50.0,116.0,207.0,1061.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24619,18442.0,2007.0,Zydrunas Ilgauskas,"$10,142,156.00",C,31.0,CLE,78.0,78.0,2130.0,...,80.7%,242.0,357.0,599.0,123.0,48.0,98.0,141.0,257.0,925.0
24620,19003.0,2008.0,Zydrunas Ilgauskas,"$10,841,615.00",C,32.0,CLE,73.0,73.0,2222.0,...,80.2%,263.0,419.0,682.0,104.0,34.0,120.0,135.0,247.0,1029.0
24621,19600.0,2009.0,Zydrunas Ilgauskas,"$11,541,074.00",C,33.0,CLE,65.0,65.0,1765.0,...,79.9%,157.0,333.0,490.0,64.0,28.0,84.0,90.0,183.0,838.0
24622,20187.0,2010.0,Zydrunas Ilgauskas,NaN,C,34.0,CLE,64.0,6.0,1339.0,...,74.3%,114.0,231.0,345.0,48.0,14.0,50.0,63.0,183.0,474.0


In [70]:
percOrigData = len(data)/len(copyData) * 100
print(f'The percentage of remaining rows left of original data is {percOrigData}')

The percentage of remaining rows left of original data is 65.48629441624365


In [73]:
# removing the columns with rows containing data from below the year 1979 since the 3pt shot wasn't introduced until
# the 1979
data = data.drop(data[data['Season Start'] < 1979.0].index)

In [75]:
print(set(data['Season Start'])) # to verify that we removed the rows that had a year that is less than 1979

{1980.0, 1981.0, 1982.0, 1983.0, 1984.0, 1985.0, 1986.0, 1987.0, 1988.0, 1989.0, 1990.0, 1991.0, 1992.0, 1993.0, 1994.0, 1995.0, 1996.0, 1997.0, 1998.0, 1999.0, 2000.0, 2001.0, 2002.0, 2003.0, 2004.0, 2005.0, 2006.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0}


In [76]:
print(len(data)) # luckily, we didn't remove any more rows from our data

16126


In [77]:
# now we need to one hot encode some of the columns since they have non-numerical data
dummyPlayName = pd.get_dummies(data['Player Name'])

In [78]:
dummyPlayName.head()

,A.C. Green,A.J. English,A.J. Guyton,A.J. Hammons,A.J. Price,A.J. Wynder,Aaron Brooks,Aaron Gordon,Aaron Gray,Aaron Harrison,...,Zach Randolph,Zan Tabak,Zarko Cabarkapa,Zarko Paspalj,Zaza Pachulia,Zeljko Rebraca,Zendon Hamilton,Zoran Dragic,Zoran Planinic,Zydrunas Ilgauskas
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
data = pd.concat([data, dummyPlayName], axis=1)
data = data.drop(columns = ['Player Name'])

NameError: name 'trainDataClean' is not defined